In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%run 1.tokenizer.ipynb

tokenizer

In [2]:
%run 2.dataset.ipynb

dataset = Dataset(negative_label=False, with_answer=False)

len(dataset), dataset[0]

(1000000, {'text': '9092+3039=12131+2715=', 'label': 1})

In [3]:
%run 3.model.ipynb

model_actor = torch.load('model/ppo', weights_only=False).to(device)

In [4]:
#====question====
question = random.choices(dataset, k=16)
question = [i['text'] for i in question]

question = tokenizer(question,
                     device=device,
                     add_bos_token=True,
                     add_eos_token=False,
                     padding_side='left')['input_ids']

#====gen====
gen = generate(model_actor,
               input_ids=question,
               pad_token_id=tokenizer.pad_token_id,
               eos_token_id=tokenizer.eos_token_id)
gen = gen[:, question.shape[1]:]

last_index = lambda lst, ele: len(lst) - lst[::-1].index(ele)

for q, g in zip(question, gen):
    q = tokenizer.decode(q)
    g = tokenizer.decode(g)

    print({'question': q, 'gen': g})

{'question': 'B1505+1349=', 'gen': '2854E'}
{'question': 'B2437+4395=', 'gen': '6832E'}
{'question': 'B4520+9682=14202+6648=', 'gen': '20850E'}
{'question': 'B1123+6282=7405+9943=', 'gen': '17348E'}
{'question': 'B4272+2422=6694+1660=', 'gen': '8354E'}
{'question': 'B1318+4655=5973+849=', 'gen': '6822E'}
{'question': 'B7902+2157=10059+9503=19562+3976=', 'gen': '23538E'}
{'question': 'B7561+6435=13996+8341=22337+3570=', 'gen': '25907E'}
{'question': 'B5523+5655=11178+1142=12320+7360=', 'gen': '19680E'}
{'question': 'B7646+2607=10253+5962=16215+6479=', 'gen': '22694E'}
{'question': 'B6170+7481=', 'gen': '13651E'}
{'question': 'B8975+7528=16503+7284=23787+7744=', 'gen': '31531E'}
{'question': 'B427+9585=10012+4876=14888+6666=', 'gen': '21554E'}
{'question': 'B5646+2024=7670+8478=16148+3802=', 'gen': '19950E'}
{'question': 'B8332+553=8885+7602=16487+1586=', 'gen': '18073E'}
{'question': 'B2801+482=3283+6315=9598+6933=', 'gen': '16539E'}


In [5]:
for i in random.choices(dataset, k=16):
    print(i['text'].count('='),i['text'])

2 1670+5429=7099+2747=
3 3924+3389=7313+8346=15659+8604=
3 2719+6277=8996+3262=12258+953=
2 826+1907=2733+5646=
3 3922+1595=5517+1165=6682+4480=
1 3701+1823=
3 5799+8308=14107+5858=19965+2942=
3 4707+9207=13914+4240=18154+8104=
1 6662+8664=
3 4694+7519=12213+6355=18568+9461=
1 1152+7609=
2 4902+4140=9042+9570=
2 537+4710=5247+6295=
3 9543+5018=14561+6831=21392+8808=
1 2630+1753=
2 7195+6947=14142+3261=


In [6]:
sent = ''


def inference(q):
    global sent

    sent += q + '='

    #限制最大轮次
    if sent.count('=') > 3:
        idx = sent.index('=') + 1
        sent = sent[idx:]

    print('sent', sent)

    #编码
    q = tokenizer([sent],
                  device=device,
                  add_bos_token=True,
                  add_eos_token=False)['input_ids']

    print('q', tokenizer.decode(q[0]))

    #生成回答
    gen = generate(model_actor,
                   input_ids=q,
                   pad_token_id=tokenizer.pad_token_id,
                   eos_token_id=tokenizer.eos_token_id)
    gen = gen[:, q.shape[1]:][0].tolist()

    #切除eos
    if tokenizer.eos_token_id in gen:
        idx = gen.index(tokenizer.eos_token_id)
        gen = gen[:idx]

    print('gen', gen)

    #如果生成了多余的对话,切除
    if tokenizer.eq_token_id in gen:
        idx = gen.index(tokenizer.eq_token_id)
        gen = gen[:idx]

    print('gen', gen)

    gen = tokenizer.decode(gen)

    print('gen', gen)

    #拼接上下文
    sent += gen

    print('sent', sent)

    return gen


inference('1000+1000')

sent 1000+1000=
q B1000+1000=
gen [6, 4, 4, 4]
gen [6, 4, 4, 4]
gen 2000
sent 1000+1000=2000


'2000'

In [7]:
inference('+500')

sent 1000+1000=2000+500=
q B1000+1000=2000+500=
gen [6, 9, 4, 4]
gen [6, 9, 4, 4]
gen 2500
sent 1000+1000=2000+500=2500


'2500'

In [8]:
inference('+500')

sent 1000+1000=2000+500=2500+500=
q B1000+1000=2000+500=2500+500=
gen [7, 4, 4, 4]
gen [7, 4, 4, 4]
gen 3000
sent 1000+1000=2000+500=2500+500=3000


'3000'

In [9]:
inference('+500')

sent 2000+500=2500+500=3000+500=
q B2000+500=2500+500=3000+500=
gen [7, 9, 4, 4]
gen [7, 9, 4, 4]
gen 3500
sent 2000+500=2500+500=3000+500=3500


'3500'